In [1]:
import pandas as pd

# Seleção de termos para suportar a análise abaixo

In [34]:
proposicoes = pd.read_csv('../dados/proposicoes_legislativas_limpas.csv')

In [40]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152419 entries, 0 to 152418
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       152419 non-null  int64 
 1   id               152419 non-null  int64 
 2   siglaTipo        152419 non-null  object
 3   ano              152419 non-null  int64 
 4   codTipo          152419 non-null  int64 
 5   descricaoTipo    152419 non-null  object
 6   ementa           152419 non-null  object
 7   ementaDetalhada  42580 non-null   object
 8   keywords         152386 non-null  object
dtypes: int64(4), object(5)
memory usage: 10.5+ MB


In [35]:
proposicoes_not_na = proposicao[~proposicao['keywords'].isnull()]

TypeError: string indices must be integers

In [36]:
keywords = proposicoes_not_na['keywords']

In [37]:
bolsa_palavras = []

In [38]:
for proposicao in keywords:
    lista = proposicao.split(',')
    bolsa_palavras.extend(lista)

In [39]:
bolsa_palavras

['manutenção',
 ' decisão',
 ' tribunal',
 ' contas',
 ' união',
 ' tcu',
 ' denegação',
 ' registro',
 ' contrato',
 ' ministério',
 ' fazenda',
 ' empresa',
 ' cimento',
 'manutenção',
 ' ato',
 ' tribunal',
 ' contas',
 ' união',
 ' tcu',
 ' recusa',
 ' registro',
 ' convenio',
 ' superintendencia',
 ' desenvolvimento',
 ' fronteira',
 ' sudoeste',
 ' sudesul',
 ' instituição',
 ' educacional',
 ' instituição',
 ' beneficente',
 ' municipio',
 ' santa',
 ' vitoria',
 ' palmar',
 ' rs',
 ' rio',
 ' grande',
 ' sul',
 'manutenção',
 ' decisão',
 ' tribunal',
 ' contas',
 ' união',
 ' tcu',
 ' denegação',
 ' registro',
 ' contrato',
 ' união',
 ' federal',
 ' enfiteuse',
 ' terreno',
 ' marinha',
 ' municipio',
 ' niteroi',
 ' rj',
 ' rio',
 ' estado',
 'manutenção',
 ' ato',
 ' tribunal',
 ' contas',
 ' união',
 ' tcu',
 ' negação',
 ' registro',
 ' contrato',
 ' superintendencia',
 ' desenvolvimento',
 ' fronteira',
 ' sudoeste',
 ' sudesul',
 ' entidade',
 ' aluguel',
 ' imovel',
 '

# Tentando tratar a caralha da falta de dados de 33 linhas 

In [2]:
df_proposicoes2 = pd.read_csv('../dados/proposicoes_legislativas_limpas.csv')

In [3]:
df_proposicoes2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152419 entries, 0 to 152418
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       152419 non-null  int64 
 1   id               152419 non-null  int64 
 2   siglaTipo        152419 non-null  object
 3   ano              152419 non-null  int64 
 4   codTipo          152419 non-null  int64 
 5   descricaoTipo    152419 non-null  object
 6   ementa           152419 non-null  object
 7   ementaDetalhada  42580 non-null   object
 8   keywords         152386 non-null  object
dtypes: int64(4), object(5)
memory usage: 10.5+ MB


In [4]:
keywords = df_proposicoes2['keywords']
indice = df_proposicoes2['Unnamed: 0']

In [207]:
keywords[:5]

0    manutenção, decisão, tribunal, contas, união, ...
1    manutenção, ato, tribunal, contas, união, tcu,...
2    manutenção, decisão, tribunal, contas, união, ...
3    manutenção, ato, tribunal, contas, união, tcu,...
4    manutenção, ato, tribunal, contas, união, tcu,...
Name: keywords, dtype: object

In [208]:
indice[:5]

0    105
1    106
2    107
3    108
4    109
Name: Unnamed: 0, dtype: int64

In [209]:
tipos = []
for item in keywords:
    tipo = type(item)
    tipos.append(tipo)

In [210]:
set(tipos)

{float, str}

In [211]:
indice_keywords = zip(indice, keywords)

In [212]:
len(list(indice_keywords))

152419

In [216]:
for indice, keywords in zip(indice, keywords):
    valor = type(keywords)
    tipos_keywords[indice] = valor

In [218]:
proposicoes_float = []

In [219]:
for chave, valor in tipos_keywords.items():
    if valor == float:
        proposicoes_float.append(chave)

In [220]:
len(proposicoes_float)

33

In [221]:
proposicoes_float

[70807,
 13965,
 59485,
 59590,
 59616,
 59626,
 59736,
 112544,
 157499,
 158709,
 158757,
 158997,
 158998,
 196945,
 434259,
 489979,
 494540,
 497676,
 497685,
 497688,
 499202,
 500299,
 500301,
 504326,
 507643,
 507644,
 507646,
 571962,
 571967,
 574600,
 680349,
 680353,
 680354]

In [114]:
for proposicao in proposicoes_float:
    print(keywords[proposicao])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


# Houve aumento de propostas legislativas cuja temática beneficia grupos historicamente marginalizados (XXXXX DESCREVA AQUI ALGUNS DOS GRUPOS APÓS ESCOLHER AS PALAVRAS CHAVES A SEREM ANALISADAS XXX) no período entre 1933 e 2021?
**Hipótese:** XXXXXX